In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.signal import resample
from typing import Literal

# Per Person Analysis

In [2]:
def parse_stimuli(df: pd.DataFrame):
    stimuli_val = df.value.str.split("-")
    df["modality"] = stimuli_val.map(lambda x: x[0])
    df["word"] = stimuli_val.map(lambda x: x[1])
    del df['value']
    return df

def load_data(experiment: str):
    attention = pd.read_csv(f'data/{experiment}/attention.csv', parse_dates=["timestamp"], index_col=["timestamp"])
    raw = pd.read_csv(f'data/{experiment}/raw-values.csv', parse_dates=["timestamp"], index_col=["timestamp"])

    stimuli = pd.read_csv(f'data/{experiment}/stimuli.csv', parse_dates=["timestamp"], index_col=["timestamp"])
    stimuli = parse_stimuli(stimuli)
    
    return stimuli, attention, raw

In [3]:
def filter_trials(stimuli_df: pd.DataFrame, modality: Literal['SAY', 'THINK']):
    return stimuli_df[stimuli_df.modality == modality]

In [4]:
MS = pd.Timedelta(1, 'ms')
def clip_trial(df: pd.DataFrame, trial_onset: pd.Timestamp):
    df = df[trial_onset - (1000 * MS):trial_onset + (2000 * MS)]
    return df

In [14]:
palette = {'YES': '#44546A', 'STOP': '#FF7C80' }
def plot_avg_trial(stimuli: pd.DataFrame, values: pd.DataFrame, do_resample = False):
    trials = [clip_trial(values, x).reset_index(drop=True) for x in stimuli.index]
    if do_resample:
        resample_len = min([x.shape[0] for x in trials]) // 2
        trials = [
            pd.Series(resample(x['value'], resample_len))
            for x in trials
        ]
    trials = pd.concat(trials, axis=1, ignore_index=True).T
    trials = trials.set_index(stimuli.word)
    trials = trials.stack().reset_index()
    plt = sns.lineplot(data=trials, x='level_1', y=0, hue='word', palette=palette)
    return plt

<hr />

In [15]:
def plot_experiment(experiment: str):
    stimuli, attention, raw = load_data(experiment)
    
    # == YES/STOP SPOKEN TRIALS ==
    df = filter_trials(stimuli, 'SAY')

    # Raw
    fig = plot_avg_trial(df, raw, True)
    fig.set(xlabel='Observation', ylabel='Power', title='Spoken Raw EEG Power')
    plt.savefig(f'{experiment}-eeg-spoken.png')
    plt.clf()

    # Attention
    fig = plot_avg_trial(df, attention)
    fig.set(xlabel='Observation', ylabel='Strength', title='Spoken Neurolink Attention Strength')
    plt.savefig(f'{experiment}-attention-spoken.png')
    plt.clf()

    # == YES/STOP IMAGINED TRIALS ==
    df = filter_trials(stimuli, 'THINK')

    # Raw
    fig = plot_avg_trial(df, raw, True)
    fig.set(xlabel='Observation', ylabel='Power', title='Imagined Raw EEG Power')
    plt.savefig(f'{experiment}-eeg-imagined.png')
    plt.clf()

    # Attention
    fig = plot_avg_trial(df, attention)
    fig.set(xlabel='Observation', ylabel='Strength', title='Imagined Neurolink Attention Strength')
    plt.savefig(f'{experiment}-attention-imagined.png')
    plt.clf()

<hr />

In [17]:
for experiment in [f'subject-{x}' for x in range(1, 8)]:
    plot_experiment(experiment)

<Figure size 432x288 with 0 Axes>

# Large Analysis

In [42]:
combined = [load_data(experiment) for experiment in [f'subject-{x}' for x in range(1, 8)]]
stimuli = pd.concat([x[0] for x in combined], axis=0).sort_index()
attention = pd.concat([x[1] for x in combined], axis=0).sort_index()
raw = pd.concat([x[2] for x in combined], axis=0).sort_index()

In [43]:
# == YES/STOP SPOKEN TRIALS ==
df = filter_trials(stimuli, 'SAY')

# Raw
fig = plot_avg_trial(df, raw, True)
fig.set(xlabel='Observation', ylabel='Power', title='Spoken Raw EEG Power')
plt.savefig(f'global-eeg-spoken.png')
plt.clf()

# Attention
fig = plot_avg_trial(df, attention)
fig.set(xlabel='Observation', ylabel='Strength', title='Spoken Neurolink Attention Strength')
plt.savefig(f'global-attention-spoken.png')
plt.clf()

# == YES/STOP IMAGINED TRIALS ==
df = filter_trials(stimuli, 'THINK')

# Raw
fig = plot_avg_trial(df, raw, True)
fig.set(xlabel='Observation', ylabel='Power', title='Imagined Raw EEG Power')
plt.savefig(f'global-eeg-imagined.png')
plt.clf()

# Attention
fig = plot_avg_trial(df, attention)
fig.set(xlabel='Observation', ylabel='Strength', title='Imagined Neurolink Attention Strength')
plt.savefig(f'global-attention-imagined.png')
plt.clf()